# Setup

In [1]:
# check GPU
!nvidia-smi

Fri Dec 30 21:38:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# pulls repo to colab
!git clone https://github.com/char-tan/mode_connectivity.git

# checkout specific branch if needed
!cd mode_connectivity; git checkout geodesics

Cloning into 'mode_connectivity'...
remote: Enumerating objects: 402, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 402 (delta 106), reused 125 (delta 85), pack-reused 222
Receiving objects: 100% (402/402), 7.12 MiB | 24.45 MiB/s, done.
Resolving deltas: 100% (225/225), done.
Already on 'main'
Your branch is up to date with 'origin/main'.


In [4]:
# allows you to change the .py files and have the imports updated
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
SCRIPT_DIR = os.path.dirname(os.path.abspath('.'))
sys.path.append(os.path.dirname(SCRIPT_DIR))

sys.path.append("mode_connectivity")

from mode_connectivity.training import *
from mode_connectivity.lmc import *
from mode_connectivity.training_config import *

from mode_connectivity.models.mlp import MLP
from mode_connectivity.models.vgg import VGG
from mode_connectivity.models.resnet import ResNet

from mode_connectivity.utils import weight_matching, data, training_utils, plot, utils

import torch
from torchvision import transforms
import copy
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'utils'

In [ ]:
from models.mlp import MLP
from utils.metrics import JSD_loss
from utils.data import get_data_loaders
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from lmc import model_interpolation
from mode_connectivity.geodesic_opt import *

weights_a = torch.load("model_files/model_a.pt", map_location=torch.device('cpu'))
weights_b = torch.load("model_files/model_b.pt", map_location=torch.device('cpu'))
weights_bp = torch.load("model_files/permuted_model_b.pt", map_location=torch.device('cpu'))

trainset = datasets.MNIST(
    root="utils/data", train=True, download=True,
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))

dl = DataLoader(trainset, 64)

# batch = next(enumerate(dl))
# idx, (batch_imgs, img_labels) = batch

# %% (A cell for testing optimise_geodesic))

opt_weights, losses = optimise_for_geodesic(
    MLP, weights_a, weights_b,
    n = 10,
    loss_metric = JSD_loss,
    dataloader = dl,
    max_iterations = 99,
    learning_rate = 0.1,
    return_losses=True
)

plt.plot(losses)

# %% (A cell for testing model_geodesic_interpolation)
model_a, model_b = MLP(), MLP()
model_a.load_state_dict(weights_a)
model_b.load_state_dict(weights_bp)
train_loader, test_loader = get_data_loaders(
    dataset="mnist", train_kwargs={"batch_size":512}, test_kwargs={"batch_size":512}
)
device, device_kwargs = get_device() # what are device_kwargs ???
losses_along_path = losses_over_geodesic(
    MLP, model_a, model_b, train_loader, test_loader, device, n_points=25,
    loss_metric=JSD_loss, max_iterations=99
)

plt.plot(losses_along_path[0])
plt.plot(losses_along_path[1])



In [ ]:
# %% (Cell for testing plot_losses_over_geodesic)
model_a, model_b = MLP(), MLP()
model_a.load_state_dict(weights_a)
model_b.load_state_dict(weights_bp)
train_loader, test_loader = get_data_loaders(
    dataset="mnist", train_kwargs={"batch_size":512}, test_kwargs={"batch_size":512}
)
device, device_kwargs = get_device() # what are device_kwargs ???
plot_losses_over_geodesic(
    MLP, model_a, model_b, train_loader, test_loader, device, n_points=10,
    loss_metric=JSD_loss, max_iterations=10
)